In [ ]:
from msal import PublicClientApplication

CLIENT_ID = "dd395d91-6cf3-421c-a1c6-89f972ae204d"
TENANT_ID = "d44c31fd-3401-417a-9157-d7de456985e8"
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"

# 你需要的 delegated scope（通常不是 .default）
SCOPE = ["User.ReadBasic.All","Files.ReadWrite.All","Sites.ReadWrite.All"]  # 替換成你要的權限

app = PublicClientApplication(
    client_id=CLIENT_ID,
    authority=AUTHORITY
)

# 啟動 device code flow
flow = app.initiate_device_flow(scopes=SCOPE)

if "user_code" not in flow:
    raise Exception("Failed to initiate device flow")

print("請在瀏覽器開啟此網址並輸入代碼完成登入：")
print(flow["verification_uri"])
print("代碼是：", flow["user_code"])

# 等待使用者登入
result = app.acquire_token_by_device_flow(flow)

# 顯示 access token
if "access_token" in result:
    print("Access token:", result["access_token"])
    token = result["access_token"]
else:
    print("Error:", result.get("error_description"))


In [2]:
import os
import django

# Set the DJANGO_SETTINGS_MODULE environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'graph_tutorial.settings')

# Initialize Django
django.setup()

In [3]:
from tutorial.graph_helper import GraphSharePointClient
SP = GraphSharePointClient(token)

In [4]:
from asgiref.sync import sync_to_async


In [5]:

# Call the scan_routine method in a synchronous context
await sync_to_async(SP.scan_routine)()

In [6]:
await sync_to_async(SP.polling_task_pool)()

✅ Updated
📝 Replied content written for task Pre-DS discuss
✅ Updated
📝 Replied content written for task (Optional) DS meeting 


In [ ]:
SP._build_excel_range_url(objects[0].sheet_name, objects[0].field_address)

In [ ]:
objects[0].sheet_name, objects[0].field_address

In [ ]:
SP.model.objects.all()

In [ ]:
# Define a function to fetch the queryset
def fetch_all_objects():
    return list(SP.model.objects.all())

# Use sync_to_async to call the function
objects = await sync_to_async(fetch_all_objects)()

In [ ]:
objects[0].msg_id

In [ ]:
from collections import defaultdict


In [ ]:
chat_groups = defaultdict(list)
# 將每個 item 的完整資訊加入對應的 chat_groups
for item in objects:
    chat_groups[item.teams_group_id].append({
        "uuid": item.uuid,
        "owner_id": item.owner_id,
        "msg_id": item.msg_id,
        "task": item.task,
        "reason":item.reason
    })

In [ ]:
chat_groups.items()

In [ ]:
from bs4 import BeautifulSoup

def _search_message_reference(messages, user_id, msg_id):
    """
    Search from cached messages list (not API call) for a reply that:
    - is from the given user
    - is a messageReference type
    - references the given msg_id
    """
    for message in messages:
        attachments = message.get("attachments", [])
        if not attachments:
            continue

        if (
            message.get("from", {}).get("user", {}).get("id") == user_id and
            attachments[0].get("contentType") == "messageReference" and
            attachments[0].get("id") == msg_id
        ):
            soup = BeautifulSoup(message['body']['content'], "html.parser")

            text = soup.get_text(separator=' ', strip=True)
            return text

    return None

In [ ]:
for chat_id, items in chat_groups.items():
            print(items)
            try:
                print(chat_id)
                messages = SP.list_msg_in_chats(chat_id)
            except Exception as e:
                print(f"⚠️ Failed to fetch messages for chat {chat_id}: {e}")
                continue

            # 4. Search for replies matching user_id and msg_id in current chat
            for item in items:
                print(item)
                try:
                    user_id = item['owner_id']
                    print(user_id)
                    for mid in item['msg_id']:
                        print(mid)
                        content = _search_message_reference(messages, user_id, mid)
                        if content:
                            print(item['uuid'])
                            print(content)
                            print(f"📝 Replied content written for task {item['task']}")
                            break  # only process first found reply
                except Exception as e:
                    print(f"❌ Error processing task {item.task}: {e}")


In [ ]:
item['task']

In [ ]:
user_id,mid

In [ ]:
_search_message_reference(messages, user_id, mid)

In [ ]:
data = await sync_to_async(SP.model.objects.get)(uuid = item['uuid'])

In [ ]:
data.field_address